In [1]:
from PyLQR.sim import KDLRobot
from PyLQR.system import PosOrnPlannerSys, PosOrnKeypoint
from PyLQR.solver import BatchILQRCP, BatchILQR, ILQRRecursive
from PyLQR.utils import primitives, PythonCallbackMessage

import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

import logging
import sys

from scipy.spatial.transform import Rotation 

from cv_bridge import CvBridge, CvBridgeError

from utils.camera_utils import RealCamera, RealCameraROS
from utils.transform_utils import *
from utils.iLQR_wrapper import iLQR
from utils.visualisation_utils import depth2pc
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action, get_camera_pose, gridRegistrator

import argparse
from scipy.spatial.transform import Rotation

import time

from contact_grasp.srv import contactGraspnetPointcloud2, contactGraspnetPointcloud2Response

import json
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointField, CameraInfo, Image, PointCloud2
from std_msgs.msg import Header
from geometry_msgs.msg import PoseArray, Pose
import message_filters

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
bridge = CvBridge()
try:
    rospy.init_node("python_node",anonymous=True)
except:
    print("rospy already initialized")

# dispose_pos = np.array([dim, 0.66, 0.1])
# dispose_orn_wxyz = np.array([0, 1, 0.35, 0])

camera_connexion = "pyWrapper"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    intrinsic, distortion = camera.getIntrinsic()
    #retrieve image and depth to initialise camera, otherwise image is very dark
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

In [4]:
def draw_triangle(dim=0.1, color=[1, 0, 0]):
    halfdim=dim/2
    points = [
        [0, 0, 0],
        [-halfdim, -halfdim, dim],
        [halfdim, -halfdim, dim],
        [halfdim, halfdim, dim],
        [-halfdim, halfdim, dim],

    ]
    lines = [
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 1],

    ]
    colors = [color for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

ls = draw_triangle()
o3d.visualization.draw_geometries([ls])

In [5]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views_pos = json.load(json_file)

pos_dif = 1000
keys = views_pos.keys()
angle_range = [-140, 40]
pc_fused = []
pc_colors_fused = []
reference_pose = np.eye(4)
plot_coordinate = []
# while (not detected or not detected_with_collision):
horizon = 30

rgb_cv, depth_cv, depth_scale = camera.get_rgb_depth()
depth_cv = depth_cv * depth_scale

pc_fused, pc_colors_fused = depth2pc(depth_cv, intrinsic, rgb_cv)
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, 
                                            downsampling_method="voxel", voxel_size=0.005,
                                            outlier_filtering_method="radius", radius_param_arg=[25, 0.015])

coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
coordinate_line = draw_triangle(color=[1,0,0])

# coordinate_line.transform(current_pose)
# coordinate.transform(current_pose)
plot_coordinate.append(coordinate)
plot_coordinate.append(coordinate_line)

print(plot_coordinate)
pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused[:, [2,1,0]]/255)
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
# coordinate.translate([0,0 , 0.4])
pc_o3d.transform(reference_pose)
to_draw = plot_coordinate.copy()
to_draw.append(pc_o3d)
o3d.visualization.draw_geometries(to_draw)

Downsampling time: 0.7588727474212646s, Filtering time: 0.15926313400268555s
[TriangleMesh with 1134 points and 2240 triangles., LineSet with 8 lines.]


In [6]:
# from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS

bridge = CvBridge()
try:
    rospy.init_node("python_node",anonymous=True)
except:
    print("rospy already initialized")

bgr = cv2.cvtColor(rgb_cv, cv2.COLOR_RGB2BGR)
#load cv image
# img = cv2.imread("/home/vdrame/bgr.png")

#convert to ros image
ros_img = bridge.cv2_to_imgmsg(bgr, encoding="bgr8")

#call service
# rospy.wait_for_service("segmentation")

try:
    segmentation = rospy.ServiceProxy("segmentation", segmentationSrv)
    resp = segmentation(ros_img)
except rospy.ServiceException as e:
    print(e)

#convert to cv image
segmap = bridge.imgmsg_to_cv2(resp.image, desired_encoding="passthrough")
print(np.unique(segmap))
#show image
cv2.imshow("segmented image", segmap)
cv2.imshow("rgb", rgb_cv)
key=cv2.waitKey(0)

cv2.destroyAllWindows()
    

[0. 1. 2.]


QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to tar

In [60]:
transform2 = np.eye(4)
transform2[0:3, 3] = [0., -0.2, 0.4]
transform2[0:3, 0:3] = Rotation.from_euler('XYZ', [-60, 0, 0], degrees=True).as_matrix()

pc_o3d = o3d.geometry.PointCloud()
pc_o3d.points = o3d.utility.Vector3dVector(pc_ROI)
pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_ROI[:, [2,1,0]]/255)
coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
coordinate2 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
coordinate2.transform(transform2)
# coordinate.translate([0,0 , 0.4])
pc_o3d.transform(reference_pose)
to_draw = plot_coordinate.copy()
to_draw.append(pc_o3d)
to_draw.append(coordinate2)
o3d.visualization.draw_geometries(to_draw)

In [59]:
import time 
start = time.time()
def get_ROI_box(pc, pc_colors, depth, rgb, segmap, intrinsic, border_size = 0.05):

    pc_seg, pc_colors_seg = depth2pc(depth, intrinsic, rgb, segmap=segmap)

    min_bound = np.min(pc_seg, axis=0) - border_size
    max_bound = np.max(pc_seg, axis=0) + border_size

    bounding_point = np.array([[min_bound[0], min_bound[1], min_bound[2], 1],
                            [min_bound[0], min_bound[1], max_bound[2], 1],
                            [min_bound[0], max_bound[1], min_bound[2], 1],
                            [min_bound[0], max_bound[1], max_bound[2], 1],
                            [max_bound[0], min_bound[1], min_bound[2], 1],
                            [max_bound[0], min_bound[1], max_bound[2], 1],
                            [max_bound[0], max_bound[1], min_bound[2], 1],
                            [max_bound[0], max_bound[1], max_bound[2], 1]])
    #crop point cloud
    ROI_mask = np.array((pc[:,0] > min_bound[0]) & (pc[:,0] < max_bound[0]) &
                    (pc[:,1] > min_bound[1]) & (pc[:,1] < max_bound[1]) &
                    (pc[:,2] > min_bound[2]) & (pc[:,2] < max_bound[2]))

    pc_ROI = pc[ROI_mask]
    pc_colors_ROI = pc_colors[ROI_mask]

    return pc_ROI, pc_colors_ROI, bounding_point

pc_ROI, pc_colors_ROI, bounding_point = get_ROI_box(pc_fused, pc_colors_fused, depth_cv, rgb_cv, segmap, intrinsic)

def project_pc(pc, pc_colors, intrinsic, extrinsic, image_size=(720, 1280)):
    pc = np.concatenate((pc, np.ones((pc.shape[0], 1))), axis=1)

    T = np.eye(4)
    #invert transformation
    T[:3, 3] = -extrinsic[:3, :3].T @ extrinsic[:3, 3]
    T[:3, :3] = extrinsic[:3, :3].T
    #apply transformation
                    
    pc = T @ pc.T
    uv = intrinsic @ pc[:3, :]
    uv = uv / uv[2, :]
    uv = uv[:2, :].T
    uv = np.round(uv).astype(np.int32)

    #select point in image
    mask = (uv[:, 0] >= 0) & (uv[:, 0] < image_size[1]) & (uv[:, 1] >= 0) & (uv[:, 1] < image_size[0])
    image = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)

    #draw points on image
    image[uv[mask, 1], uv[mask, 0]] = pc_colors[mask][:, [0, 1, 2]]

    #draw circle on image
    for i in range(uv.shape[0]):
        if mask[i]:
            cv2.circle(image, (uv[i, 0], uv[i, 1]), round(max(image_size)/200), pc_colors[i], -1)

    return image

def apply_ROI_box_mask(image, bounding_point, intrinsic, extrinsic, image_size=(720, 1280)):
    T = np.eye(4)
    #invert transformation
    T[:3, 3] = -extrinsic[:3, :3].T @ extrinsic[:3, 3]
    T[:3, :3] = extrinsic[:3, :3].T

    #apply transformation
    bounding_point_uv = T @ bounding_point.T
    bounding_point_uv = intrinsic @ bounding_point_uv[:3, :]
    bounding_point_uv = bounding_point_uv / bounding_point_uv[2, :]
    bounding_point_uv = bounding_point_uv[:2, :].T
    bounding_point_uv = bounding_point_uv.astype(np.int32)
    mask_bounding = (bounding_point_uv[:, 0] >= 0) & (bounding_point_uv[:, 0] < image_size[1]) & (bounding_point_uv[:, 1] >= 0) & (bounding_point_uv[:, 1] < image_size[0])
    max_bound = np.max(bounding_point_uv, axis=0)
    min_bound = np.min(bounding_point_uv, axis=0)
    print(min_bound, max_bound)
    if min_bound[0] < 0:
        min_bound[0] = 0
    if min_bound[1] < 0:
        min_bound[1] = 0
    if max_bound[0] < 0:
        max_bound[0] = 0
    if max_bound[1] < 0:
        max_bound[1] = 0
    print(min_bound, max_bound)
    mask_bounding_ROI = np.ones_like(image)
    mask_bounding_ROI[min_bound[1]:max_bound[1], min_bound[0]:max_bound[0]] = 0

    image[mask_bounding_ROI.astype(np.bool)] = 255

    return image

start = time.time()
transform = transform2
image = project_pc(pc_ROI, pc_colors_ROI, intrinsic, transform, image_size=(720, 1280))
image = apply_ROI_box_mask(image, bounding_point, intrinsic, transform, image_size=(720, 1280))
print(time.time() - start)
print("score is :", (image.shape[0] * image.shape[1] * image.shape[2])/np.sum(image == 0))
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[-280  -52] [649 753]
[0 0] [649 753]
0.021442890167236328
score is : 3.7456664313883814


/home/vdrame/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move 

In [45]:
# transform = np.array([[1, 0, 0, 0],
#                       [0, 1, 0, 0],
#                       [0, 0, 1, 0],
#                       [0, 0, 0, 1]])

T = transform2.copy()

start = time.time()
## project pc on image plane
pc = np.asarray(pc_o3d.points) 

#add a 1 to the end of each point
pc = np.concatenate((pc, np.ones((pc.shape[0], 1))), axis=1)

#invert transformation
T[:3, 3] = -T[:3, :3].T @ T[:3, 3]
T[:3, :3] = T[:3, :3].T
#apply transformation

T=np.eye(4) 
pc = T @ pc.T
uv = intrinsic @ pc[:3, :]
uv = uv / uv[2, :]
uv = uv[:2, :].T
uv = np.round(uv).astype(np.int32)

img_dim = rgb.shape[:2]

#select point in image
mask = (uv[:, 0] >= 0) & (uv[:, 0] < img_dim[1]) & (uv[:, 1] >= 0) & (uv[:, 1] < img_dim[0])

bounding_point_uv = T @ bounding_point.T
bounding_point_uv = intrinsic @ bounding_point_uv[:3, :]
bounding_point_uv = bounding_point_uv / bounding_point_uv[2, :]
bounding_point_uv = bounding_point_uv[:2, :].T
bounding_point_uv = bounding_point_uv.astype(np.int32)
mask_bounding = (bounding_point_uv[:, 0] >= 0) & (bounding_point_uv[:, 0] < img_dim[1]) & (bounding_point_uv[:, 1] >= 0) & (bounding_point_uv[:, 1] < img_dim[0])
max_bound = np.max(bounding_point_uv, axis=0)
min_bound = np.min(bounding_point_uv, axis=0)
if min_bound[0] < 0:
    min_bound[0] = 0
if min_bound[1] < 0:
    min_bound[1] = 0
print(min_bound, max_bound)
image = np.zeros((img_dim[0], img_dim[1], 3), dtype=np.uint8)

#draw ROI
mask_bounding_ROI = np.ones_like(image)
mask_bounding_ROI[min_bound[1]:max_bound[1], min_bound[0]:max_bound[0]] = 0

image[mask_bounding_ROI.astype(np.bool)] = 255

#draw points on image
image[uv[mask, 1], uv[mask, 0]] = pc_colors_ROI[mask][:, [0, 1, 2]]

#draw circle on image
for i in range(uv.shape[0]):
    if mask[i]:
        cv2.circle(image, (uv[i, 0], uv[i, 1]), round(max(img_dim)/200), pc_colors_ROI[i], -1)

for i in range(bounding_point_uv.shape[0]):
    if mask_bounding[i]:
        cv2.circle(image, (bounding_point_uv[i, 0], bounding_point_uv[i, 1]), round(max(img_dim)/50), (255, 0, 0), -1)

print(time.time() - start)
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[189 392] [650 816]
0.040840864181518555


/home/vdrame/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move to target thread (0x55b52d647e70)

QObject::moveToThread: Current thread (0x55b52d647e70) is not the object's thread (0x55b5275ec750).
Cannot move 